In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import cv2
import imgproc
from PIL import Image


In [2]:
# Korekcja perspektywy
def perspective_projection(original, params):
    # Transformacja rzutowa zbioru punktów

    x, y=original
    A, B, C, D, E, F, G, H = params

    xp=(A*x+B*y+C)/(G*x+H*y+1)
    yp=(D*x+E*y+F)/(G*x+H*y+1)

    return xp, yp

In [3]:
# Obliczanie punktów
def find_transformation(points, ref_points):
    M=np.zeros((8,8))
    v=np.zeros((8))

    for i in range(0,4):
        x,y=ref_points[i]
        xp, yp=points[i]

        M[2*i, :] = np.array([x, y, 1, 0, 0, 0, -x*xp, -y*xp])
        M[2*i+1, :] = np.array([0, 0, 0, x, y, 1, -x*yp, -y*yp])

        v[2*i]=xp
        v[2*i+1]=yp

    params = np.linalg.solve(M, v)
    return params

In [4]:
def correct_perspective(img, params, width, height):
    rows=np.linspace(0, height-1, height)
    cols=np.linspace(0, width-1, width)

    x, y = np.meshgrid(cols, rows)
    x= x.astype(np.int32)
    y= y.astype(np.int32)

    xp, yp= perspective_projection((x, y), params)
    org_cols= np.floor(xp).astype(np.int32)
    org_rows= np.floor(yp).astype(np.int32)

    results=np.zeros((height, width, 3))
    results[y, x, :] = img[org_rows, org_cols, :]

    results=results.astype(np.uint8)

    return results


In [6]:
points=[]
img=cv2.imread("foto.jpg")
img_cpy=img.copy()

cv2.imshow("image", img_cpy)
cv2.imwrite("demo.jpg",img)

def add_point(event, x, y, flags, param):
    if event== cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        cv2.circle(img_cpy, (x, y), 3, [0,0,225], -1)

        if len(points) == 4:
            height = 640
            width=480
            ref_points= [(0,0), (width-1, 0), (width-1, height-1), (0, height-1)]
            params=imgproc.find_transformation(points, ref_points )
            rectified_img = imgproc.correct_perspective(img, params, width, height)
            cv2.imshow("rectified image", rectified_img)
            cv2.imwrite("correct.jpg", rectified_img)




cv2.setMouseCallback("image", add_point)

while True:
    cv2.imshow("image", img_cpy)
    key=cv2.waitKey(1)
    if key==ord("q"):
        break